# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 14 2022 12:41PM,253105,10,0086249122,ISDIN Corporation,Released
1,Dec 14 2022 12:41PM,253105,10,0086249481,ISDIN Corporation,Released
2,Dec 14 2022 12:35PM,253103,10,Yusen-8902327,Yusen – 3D Matrix,Released
3,Dec 14 2022 12:35PM,253103,10,Yusen-8902329,Yusen – 3D Matrix,Released
4,Dec 14 2022 12:35PM,253103,10,Yusen-8902343,Yusen – 3D Matrix,Released
5,Dec 14 2022 12:28PM,253100,12,HH-38750,Hush Hush,Released
6,Dec 14 2022 12:28PM,253100,12,HH-38755,Hush Hush,Released
7,Dec 14 2022 12:28PM,253100,12,HH-38759,Hush Hush,Released
8,Dec 14 2022 12:20PM,253099,19,N17884,"NAPP Technologies, LLC",Released
9,Dec 14 2022 12:00PM,253097,19,60018269,"GUSA Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,253097,Released,1
27,253099,Released,1
28,253100,Released,3
29,253103,Released,3
30,253105,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253097,NaN,NaN,1.0
253099,NaN,NaN,1.0
253100,NaN,NaN,3.0
253103,NaN,NaN,3.0
253105,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253003,0.0,0.0,10.0
253035,0.0,0.0,1.0
253051,5.0,0.0,10.0
253053,14.0,1.0,10.0
253060,4.0,10.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253003,0,0,10
253035,0,0,1
253051,5,0,10
253053,14,1,10
253060,4,10,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253003,0,0,10
1,253035,0,0,1
2,253051,5,0,10
3,253053,14,1,10
4,253060,4,10,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253003,,,10
1,253035,,,1
2,253051,5,,10
3,253053,14,1,10
4,253060,4,10,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 14 2022 12:41PM,253105,10,ISDIN Corporation
2,Dec 14 2022 12:35PM,253103,10,Yusen – 3D Matrix
5,Dec 14 2022 12:28PM,253100,12,Hush Hush
8,Dec 14 2022 12:20PM,253099,19,"NAPP Technologies, LLC"
9,Dec 14 2022 12:00PM,253097,19,"GUSA Granules USA, Inc."
10,Dec 14 2022 11:25AM,253093,19,"AdvaGen Pharma, Ltd"
12,Dec 14 2022 11:14AM,253087,10,Eywa Pharma Inc.
37,Dec 14 2022 11:09AM,253090,15,"Alliance Pharma, Inc."
80,Dec 14 2022 11:08AM,253092,10,Emerginnova
81,Dec 14 2022 11:07AM,253091,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 14 2022 12:41PM,253105,10,ISDIN Corporation,,,2
1,Dec 14 2022 12:35PM,253103,10,Yusen – 3D Matrix,,,3
2,Dec 14 2022 12:28PM,253100,12,Hush Hush,,,3
3,Dec 14 2022 12:20PM,253099,19,"NAPP Technologies, LLC",,,1
4,Dec 14 2022 12:00PM,253097,19,"GUSA Granules USA, Inc.",,,1
5,Dec 14 2022 11:25AM,253093,19,"AdvaGen Pharma, Ltd",,2,
6,Dec 14 2022 11:14AM,253087,10,Eywa Pharma Inc.,,,25
7,Dec 14 2022 11:09AM,253090,15,"Alliance Pharma, Inc.",,,43
8,Dec 14 2022 11:08AM,253092,10,Emerginnova,,,1
9,Dec 14 2022 11:07AM,253091,15,"Virtus Pharmaceuticals, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 12:41PM,253105,10,ISDIN Corporation,2,,
1,Dec 14 2022 12:35PM,253103,10,Yusen – 3D Matrix,3,,
2,Dec 14 2022 12:28PM,253100,12,Hush Hush,3,,
3,Dec 14 2022 12:20PM,253099,19,"NAPP Technologies, LLC",1,,
4,Dec 14 2022 12:00PM,253097,19,"GUSA Granules USA, Inc.",1,,
5,Dec 14 2022 11:25AM,253093,19,"AdvaGen Pharma, Ltd",,2,
6,Dec 14 2022 11:14AM,253087,10,Eywa Pharma Inc.,25,,
7,Dec 14 2022 11:09AM,253090,15,"Alliance Pharma, Inc.",43,,
8,Dec 14 2022 11:08AM,253092,10,Emerginnova,1,,
9,Dec 14 2022 11:07AM,253091,15,"Virtus Pharmaceuticals, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 12:41PM,253105,10,ISDIN Corporation,2,,
1,Dec 14 2022 12:35PM,253103,10,Yusen – 3D Matrix,3,,
2,Dec 14 2022 12:28PM,253100,12,Hush Hush,3,,
3,Dec 14 2022 12:20PM,253099,19,"NAPP Technologies, LLC",1,,
4,Dec 14 2022 12:00PM,253097,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 12:41PM,253105,10,ISDIN Corporation,2.0,NaN,NaN
1,Dec 14 2022 12:35PM,253103,10,Yusen – 3D Matrix,3.0,NaN,NaN
2,Dec 14 2022 12:28PM,253100,12,Hush Hush,3.0,NaN,NaN
3,Dec 14 2022 12:20PM,253099,19,"NAPP Technologies, LLC",1.0,NaN,NaN
4,Dec 14 2022 12:00PM,253097,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 12:41PM,253105,10,ISDIN Corporation,2.0,0.0,0.0
1,Dec 14 2022 12:35PM,253103,10,Yusen – 3D Matrix,3.0,0.0,0.0
2,Dec 14 2022 12:28PM,253100,12,Hush Hush,3.0,0.0,0.0
3,Dec 14 2022 12:20PM,253099,19,"NAPP Technologies, LLC",1.0,0.0,0.0
4,Dec 14 2022 12:00PM,253097,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2783822,85.0,3.0,7.0
12,253100,3.0,0.0,0.0
15,1012285,65.0,1.0,19.0
16,253035,1.0,0.0,0.0
19,1518523,23.0,12.0,4.0
20,253089,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2783822,85.0,3.0,7.0
1,12,253100,3.0,0.0,0.0
2,15,1012285,65.0,1.0,19.0
3,16,253035,1.0,0.0,0.0
4,19,1518523,23.0,12.0,4.0
5,20,253089,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,85.0,3.0,7.0
1,12,3.0,0.0,0.0
2,15,65.0,1.0,19.0
3,16,1.0,0.0,0.0
4,19,23.0,12.0,4.0
5,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,85.0
1,12,Released,3.0
2,15,Released,65.0
3,16,Released,1.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,7.0,0.0,19.0,0.0,4.0,0.0
Executing,3.0,0.0,1.0,0.0,12.0,0.0
Released,85.0,3.0,65.0,1.0,23.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,7.0,0.0,19.0,0.0,4.0,0.0
1,Executing,3.0,0.0,1.0,0.0,12.0,0.0
2,Released,85.0,3.0,65.0,1.0,23.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,7.0,0.0,19.0,0.0,4.0,0.0
1,Executing,3.0,0.0,1.0,0.0,12.0,0.0
2,Released,85.0,3.0,65.0,1.0,23.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()